Introduction

In the field of data science, effective data cleaning is crucial for ensuring the accuracy and reliability of analysis and machine learning models. One of the most prevalent issues in datasets is the presence of missing values, which can arise from various factors such as incomplete data collection, human error, or technical malfunctions. Addressing missing data is essential, as neglecting to do so can lead to skewed results and compromised model performance.

This notebook presents a systematic approach to handling missing values, showcasing various methodologies employed in data cleaning. The following techniques will be explored:
1. Deletion Methods: Techniques such as listwise deletion and pairwise deletion, which involve removing records with missing values.
2. Imputation Techniques:

   2.1 Mean/Median/Mode Imputation: Replacing missing values with the mean, median, or mode of the respective feature.

   2.2 K-Nearest Neighbors (KNN) Imputation: Using the values from the nearest neighbors to fill in missing data.

   2.3 Regression Imputation: Predicting missing values based on relationships with other features in the dataset.
    
   2.4 Random Forest Imputation: Leveraging the power of ensemble learning to predict and replace missing values

3. Advanced Techniques: Multivariate Imputation by Chained Equations (MICE): A sophisticated approach that uses multiple regression models to predict missing values iteratively.

By employing these methodologies, this notebook aims to provide a comprehensive understanding of how to effectively handle missing values, thereby enhancing data quality and integrity. Ultimately, the techniques presented here will empower data practitioners to make informed decisions, leading to improved analytical outcomes and model performance.

Importing Libraries 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

Load the Titanic dataset

In [2]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


check for missing values

In [4]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

Deletion Methods

In [5]:
# Listwise deletion (dropping rows with any missing values)

df_listwise = df.dropna()

# Display the shape of the original and the modified DataFrame

print("\nShape of Original DataFrame:", df.shape)
print("Shape after Listwise Deletion:", df_listwise.shape)


Shape of Original DataFrame: (891, 15)
Shape after Listwise Deletion: (182, 15)


In [6]:
# Pairwise deletion example
# For demonstration, we'll calculate the correlation matrix using pairwise deletion
# Select only numeric columns for correlation

numeric_df = df.select_dtypes(include=['number'])  # Select only numeric columns
correlation_matrix = numeric_df.corr(method='pearson')  # Automatically handles missing values
print("\nCorrelation Matrix (using Pairwise Deletion):\n", correlation_matrix)


Correlation Matrix (using Pairwise Deletion):
           survived    pclass       age     sibsp     parch      fare
survived  1.000000 -0.338481 -0.077221 -0.035322  0.081629  0.257307
pclass   -0.338481  1.000000 -0.369226  0.083081  0.018443 -0.549500
age      -0.077221 -0.369226  1.000000 -0.308247 -0.189119  0.096067
sibsp    -0.035322  0.083081 -0.308247  1.000000  0.414838  0.159651
parch     0.081629  0.018443 -0.189119  0.414838  1.000000  0.216225
fare      0.257307 -0.549500  0.096067  0.159651  0.216225  1.000000


Imputation Techniques

In [7]:
# 2.1 Mean/Median/Mode Imputation
# for Continuous Data Handling use Mean/Median
# frist add null in Continuous Feature

np.random.seed(42)
missing_mask = np.random.rand(df.shape[0]) < 0.2
df.loc[missing_mask, 'fare'] = np.nan

# 1.1 Mean Imputation for 'fare'

df2 = df.copy()
mean_fare = df2['fare'].mean()
print("Number of missing values before Mean Imputation:", df2['fare'].isnull().sum())
df2['fare'].fillna(mean_fare, inplace=True)
print(f"Mean Imputation for 'fare' Value = {mean_fare}")
print("Number of missing values after Mean Imputation:", df2['fare'].isnull().sum())

print("*"*80)

# 1.2 Median Imputation for 'fare'

df2 = df.copy()
print("Number of missing values before Median Imputation:", df2['fare'].isnull().sum())
median_fare = df2['fare'].median()
df2['fare'].fillna(median_fare, inplace=True)
print(f"Median Imputation for 'fare' Value = {median_fare}")
print("Number of missing values after Median Imputation:", df2['fare'].isnull().sum())

Number of missing values before Mean Imputation: 203
Mean Imputation for 'fare' Value = 32.18247848837209
Number of missing values after Mean Imputation: 0
********************************************************************************
Number of missing values before Median Imputation: 203
Median Imputation for 'fare' Value = 14.45625
Number of missing values after Median Imputation: 0


In [8]:
### another way using Sklearn
# 1.1 Mean Imputation for 'fare'

df2 = df.copy()
print("Number of missing values before Mean Imputation:", df2['fare'].isnull().sum())
mean_imputer = SimpleImputer(strategy='mean')
df2['fare'] = mean_imputer.fit_transform(df2[['fare']])
print(f"Mean Imputation for 'fare' Value = {mean_imputer.statistics_[0]}")
print("Number of missing values after Mean Imputation:", df2['fare'].isnull().sum())

print("*"*80)

# 1.2 Median Imputation for 'fare'

df2 = df.copy()
print("Number of missing values before Median Imputation:", df2['fare'].isnull().sum())
med_imputer = SimpleImputer(strategy='median')
df2['fare'] = med_imputer.fit_transform(df2[['fare']])
print(f"Median Imputation for 'fare' Value = {med_imputer.statistics_[0]}")
print("Number of missing values after Median Imputation:", df2['fare'].isnull().sum())

Number of missing values before Mean Imputation: 203
Mean Imputation for 'fare' Value = 32.18247848837209
Number of missing values after Mean Imputation: 0
********************************************************************************
Number of missing values before Median Imputation: 203
Median Imputation for 'fare' Value = 14.45625
Number of missing values after Median Imputation: 0


In [9]:
# 2.1 Mean/Median/Mode Imputation
# for Discrete Data Handling use mod/Median (for numerical not Categorical )

df2 = df.copy()

# 1.1 Mode Imputation for 'deck'

print("Number of missing values before Mode Imputation for 'deck':", df2['deck'].isnull().sum())
mode_deck = df2['deck'].mode()[0]
df2['deck'].fillna(mode_deck, inplace=True)
print(f"Mode Imputation for 'deck' Value = {mode_deck}")
print("Number of missing values after Mode Imputation for 'deck':", df2['deck'].isnull().sum())

Number of missing values before Mode Imputation for 'deck': 688
Mode Imputation for 'deck' Value = C
Number of missing values after Mode Imputation for 'deck': 0


In [10]:
### another way using Sklearn

df2 = df.copy()
print("Number of missing values before Mode Imputation for 'deck':", df2['deck'].isnull().sum())
mode_imputer = SimpleImputer(strategy='most_frequent')
df2['deck'] = mode_imputer.fit_transform(df2[['deck']]).flatten() 
print(f"Mode Imputation for 'embarked' Value = {mode_imputer.statistics_[0]}")
print("Number of missing values after Mode Imputation for 'deck':", df2['deck'].isnull().sum())

Number of missing values before Mode Imputation for 'deck': 688
Mode Imputation for 'embarked' Value = C
Number of missing values after Mode Imputation for 'deck': 0


In [11]:
# 2.2 K-Nearest Neighbors (KNN) Imputation for Continuous KNN is not applicable to non-numeric data.

df2 = df.copy()
print("Missing Values before Imputation for 'fare':", df2['fare'].isnull().sum())
imputer = KNNImputer(n_neighbors=5)
df2[['fare']] = imputer.fit_transform(df2[['fare']])
print("Missing Values after Imputation for 'fare':", df2['fare'].isnull().sum())

Missing Values before Imputation for 'fare': 203
Missing Values after Imputation for 'fare': 0


Machine Learning-Based Imputation for Missing Values

In [12]:
# 2.3 Regression Imputation for Continuous use any model to predict null values

df2 = df.copy()
missing_mask = df2['fare'].isnull()

# Prepare features and target variable for training the model
# Assuming 'age', 'pclass', and 'sibsp' are useful features to predict 'fare'

features = ['parch', 'pclass', 'sibsp','survived']
df2_non_missing = df2.loc[~missing_mask, features + ['fare']].dropna()
X = df2_non_missing[features]
y = df2_non_missing['fare']

# Train the RandomForestRegressor model

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

# Predict missing values in 'fare'

X_missing = df2.loc[missing_mask, features]
predicted_fare = rf.predict(X_missing)

# Fill the missing values in 'fare' column with the predicted values

df2.loc[missing_mask, 'fare'] = predicted_fare

# Print the imputed values for verification

print("Predicted 'fare' values for missing data:")
print(predicted_fare[:10])

# Verify if missing values are filled

print("Missing Values After ML Imputation in 'fare':", df2['fare'].isnull().sum())

Predicted 'fare' values for missing data:
[ 8.79768036  8.79768036 37.40036826 15.5281786   8.79768036 13.99126981
 13.99126981  8.79768036  8.79768036 79.03317472]
Missing Values After ML Imputation in 'fare': 0


In [13]:
# 2.4 Random Forest Imputation for Discrete use any model to predict null values

df2 = df.copy()
missing_mask = df2['deck'].isnull()

# Prepare features and target variable for training the model
# Assuming 'age', 'pclass', and 'sibsp' are useful features to predict 'fare'

features = ['parch', 'pclass', 'sibsp','survived']
df2_non_missing = df2.loc[~missing_mask, features + ['deck']].dropna()
X = df2_non_missing[features]
y = df2_non_missing['deck'].astype('category').cat.codes

# Train the RandomForestClassifier model

rf_classifier  = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier .fit(X, y)

# Predict missing values in 'deck'

X_missing = df2.loc[missing_mask, features]
predicted_deck = rf_classifier .predict(X_missing)

# Fill the missing values in 'deck' column with the predicted values

df2.loc[missing_mask, 'deck'] = pd.Categorical.from_codes(predicted_deck, df2_non_missing['deck'].astype('category').cat.categories)

# Print the imputed values for verification

print("Predicted 'deck' values for missing data:")
print(df2.loc[missing_mask, 'deck'][:10].values)

# Verify if missing values are filled

print("Missing Values After ML Imputation in 'deck':", df2['deck'].isnull().sum())

Predicted 'deck' values for missing data:
['F', 'E', 'F', 'F', 'G', 'G', 'F', 'F', 'G', 'F']
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Missing Values After ML Imputation in 'deck': 0


Advanced Techniques: Multivariate Imputation by Chained Equations (MICE)

In [14]:
# Define the IterativeImputer with different models for continuous variables

df2 = df.copy()
missing_mask = df2['fare'].isnull()
imputer = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=100, random_state=42), 
    max_iter=100, 
    random_state=42
)

# Perform imputation on the continuous variables

df2['fare'] = imputer.fit_transform(df[['fare']])
print("Predicted 'fare' values for missing data:")
print(df2.loc[missing_mask, 'fare'][:10].values)

print("Missing Values After IterativeImputer in 'fare':", df2['fare'].isnull().sum())

Predicted 'fare' values for missing data:
[32.18247849 32.18247849 32.18247849 32.18247849 32.18247849 32.18247849
 32.18247849 32.18247849 32.18247849 32.18247849]
Missing Values After IterativeImputer in 'fare': 0


In [15]:
# Define the IterativeImputer with different models for Discrete variables

df2 = df.copy()
missing_mask = df2['deck'].isnull()
df2.deck = df2.deck.astype('category').cat.codes
df2.deck.replace(-1,np.NAN,inplace=True)

imputer = IterativeImputer(
    estimator=RandomForestClassifier(n_estimators=100, random_state=42), 
    max_iter=100, 
    random_state=42
)

# Perform imputation on the continuous variables

df2['deck'] = imputer.fit_transform(df2[['deck']])

# Convert the imputed 'deck' column back to categorical after imputation

df2['deck'] = pd.Categorical.from_codes(
    df2['deck'].round(0).astype(int), 
    df['deck'].astype('category').cat.categories
)
print(df2.loc[missing_mask,'deck'][:10].values)
print("Missing Values After IterativeImputer in 'deck':", df2['deck'].isnull().sum())

['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Missing Values After IterativeImputer in 'deck': 0
